# Dann5 d5o: Use of Solvers

**Dann5 d5o programming framework** is an extension of python programming langue. By generalizing D-Wave CSP implementation, it helps QAC programmers develop faster programming logic through use of constructs such as data types definitions, equation and condition statements, programmable routines, functions, and specialized functions.

Here, we are continuing an overview of d5o features by:
1.	Introducing **d5o Qsolverand Qanalyzer**. Qsolver is a local Qubo solver, an alternative to [D-Wave’s ExactSolver]( https://docs.ocean.dwavesys.com/projects/dimod/en/latest/reference/sampler_composites/samplers.html#module-dimod.reference.samplers.exact_solver).
2.	executing complex d5o equations on QAC using [D-Wave samplers]( https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html#dwavesampler); 
3.	solving a large complex d5o equation using [D-Wave hybrid-sampler]( https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html#leaphybridsampler);


If you have [installed Dann5 d5o library](https://github.com/voya-voja/dann5/blob/master/README.md) and all prerequisites, we can start.

## d5o Qsolver and Qanalyzer

From module *dann5.d5o* we will import **Qsolver and Qanalyser** in addition to *Qvar* and *Qequation* that have been used in the [d5o overview of basic features](https://github.com/voya-voja/dann5/blob/master/examples/py/d5o_basic_features.ipynb).
- *Qsolver* is a Qubo solver that runs on a local machine
- *Qanalyzer* is a functional object that analyzes Qubo. It has methods to request linear nodes() and quadratic branches() for a given Qubo

Also, we will import D-Wave's *ExactSolver* from dimod module. And, we will import *time* module to measure performances of solving Qubo's with different levels of complexity (i.e. higher number of nodes/branches) using local solvers, i.e. Qsolver and ExactSolver, remote QAC samplers, i.e. Advantage and 2000Q, and D-Wave's hybrid solver.

In [1]:
from dann5.d5o import Qvar, Qequation, Qsolver, Qanalyzer
from dimod import ExactSolver
import time

For a problem:

> If **T = 2 * a + b * c**, find all possible solutions when **T = 11** and **a, b, c <= 3**.

Like in the [d5o overview of basic features](https://github.com/voya-voja/dann5/blob/master/examples/py/d5o_basic_features.ipynb), we are using python variable **_2**, **a**, **b**, **c** and **T** to reference *Q variables* ***2_***, ***a***, ***b***, ***c*** and ***T*** respectfully, where 
- Q constant *2_* has a value **2**; 
- Q variables *a*, *b* and *c* are initialized with **2 Q bits**, i.e. limiting their values to {0, 1, 2, 3}
- Q variable *T* is set to **11**, and it is used as a Q equation result.

Python variable **eT** references the *Q equation* **T = 2_ * a + b * c**

***Note:*** parentheses around multipliers are required in python implementation, *(still to investigate why)*. In C++ implementation they are not required as *multiplication operator has precedence over addition operator*.

In [2]:
_2 = Qvar("2_",2)
a = Qvar(2, "a")
b = Qvar(2, "b")
c = Qvar(2, "c")
T = Qvar("T", 11)
eT = Qequation(T)
eT.assign((_2 * a) + (b * c))
print(eT.toString())

| T0 = 1 | = 2_0 & a0 ^ b0 & c0
| T1 = 1 | = 2_0 & a1 + 2_1 & a0 + b0 & c1 ^ b1 & c0 ^ #|T0|
| T2 = 0 | = 2_1 & a1 + #|2_0 & a1 + 2_1 & a0 + b0 & c1 ^ b1 & c0| + b1 & c1 ^ #|b0 & c1 ^ b1 & c0| ^ #|T1|
| T3 = 1 | = #|2_1 & a1 + #|2_0 & a1 + 2_1 & a0 + b0 & c1 ^ b1 & c0| + b1 & c1 ^ #|b0 & c1 ^ b1 & c0|| + #|b1 & c1 ^ #|b0 & c1 ^ b1 & c0|| + #|T2|
| T4 = 0 | = #|T3|



To better understand a complexity of the Q equation above, we are analyzing the Qubo generated for the 

In [3]:
Q = eT.qubo()
analyzer = Qanalyzer(Q)
nNo = analyzer.nodesNo()
print("number of nodes:", nNo, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 25 has 8,388,658.0 possible solutions


***Qsolver*** is a *Qubo solver* that runs on a local machine, it is optimized for solving Qubo by **generating by default sample with lowest energy**. 

To get a full sampleset use overloaded constructor when creating an instance of Qsolver:
> solver = Qsolver(Q, False);

In [4]:
start = time.process_time()
d5oSolver = Qsolver(Q)
sample = d5oSolver.solution()
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 1.828125 seconds to solve problem with 25 nodes


In [5]:
eT.set(sample)
print(eT.solutions())

2_ = 2; a = 1; b = 3; c = 3; 



As expected there is only one solution of the problem above, i.e. *a is 1 while b and c are 3*.

Now we will execute the same Qubo using ExactSolver.
> The next section of code expect **to run between 1 and 2 minutes**, if you are going to run it on Core i7 with 32GB RAM.

In [6]:
start = time.process_time()
dwaveSolver = ExactSolver()
sampleset = dwaveSolver.sample_qubo(Q)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
finish = time.process_time()
print("ExactSolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

ExactSolver takes 74.59375 seconds to solve problem with 25 nodes


In finding a sample with the lowest energy for a given Qubo, Qsolver is ~40 times faster than ExactSolver. Also, ExactSolver allocates significant amount of memory, and for Qubo's with more than 30 nodes it runs out of space on a laptop with 32GB RAM and ~200GB of free hard-disk space.

In [7]:
eT.set(sample)
print(eT.solutions())

2_ = 2; a = 1; b = 3; c = 3; 



By using d5o we can easily solve on a local machine a problem like:
> A school has received 45 tickets for a show. If there are 13 schoolteachers to oversee pupils, with a help of no more than 3 parents, what would be a similar number of boys and girls that can see the show?

In [8]:
girls = Qvar(4, "girls")
boys = Qvar(4, "boys")
teachers = Qvar("teachers", 13)
parents = Qvar(2, "parents")
tickets = Qvar("tickets", 45)
distribution = Qequation(tickets).assign( girls + boys + teachers + parents )
Q = distribution.qubo()
nNo = Qanalyzer(Q).nodesNo()
print("number of nodes:", nNo, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 28 has 67,108,920.0 posible solutions


> The next segment of code expect to run 14+ seconds.

In [9]:
start = time.process_time()
d5oSolver = Qsolver(Q)
sample = d5oSolver.solution()
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 14.3125 seconds to solve problem with 28 nodes


In [10]:
distribution.set(sample)
print(distribution.solutions())

girls = 15; boys = 14; teachers = 13; parents = 3; 
girls = 14; boys = 15; teachers = 13; parents = 3; 
girls = 15; boys = 15; teachers = 13; parents = 2; 



Similarly, *a multiplication Q equation with three Q variables, one a 3 qbit and two 2 qbits variables, will generate* ***a Qubo with 32 nodes***.

The problem below is a variation of [D-Wave's Leap factorial problem](https://cloud.dwavesys.com/learning/user/nebojsa_2evojinovic_40rogers_2ecom/notebooks/leap/demos/factoring/01-factoring-overview.ipynb), only here we are finding possible values for 3 Q variables ***x <= 7*** **and** ***y & z <= 3***, **where** ***a result P is 42***.

In [11]:
x = Qvar(3, "x")
y = Qvar(2, "y")
z = Qvar(2, "z")
P = Qvar("P", 42)
eP = Qequation(P).assign(x * y * z)
Q = eP.qubo()
analyzer = Qanalyzer(Q)
nNo = analyzer.nodesNo()
print("number of nodes:", nNo, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 32 has 1,073,741,888.0 posible solutions


> On a laptop (Core i7 & 32GB RAM) Qubo with 32 nodes will require a bit less than **5 minutes of computation**.

In [12]:
start = time.process_time()
d5oSolver = Qsolver(Q)
sample = d5oSolver.solution()
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 288.171875 seconds to solve problem with 32 nodes


In [13]:
eP.set(sample)
print(eP.solutions())

x = 7; y = 3; z = 2; 
x = 7; y = 2; z = 3; 



## d5o equations on QAC using D-Wave samplers

When defining Q variables to solve a problem, it is important to ensure that there is a possible solution within defined Q equation. For example, 
> if we would like *to rerun the above school trip example for 78 available tickets*, **and we do not adjust variables**, ***we will get a result set that does not make any sense***:

In [14]:
tickets = Qvar("tickets", 78)
distribution = Qequation(tickets).assign( girls + boys + teachers + parents )
Q = distribution.qubo()
d5oSolver = Qsolver(Q)
sample = d5oSolver.solution()
distribution.set(sample)
print(distribution.solutions())

girls = 0; boys = 0; teachers = 13; parents = 1; 
girls = 1; boys = 0; teachers = 13; parents = 0; 
girls = 0; boys = 1; teachers = 13; parents = 0; 



Obviously we need to adjust *the size of ***girls and boys*** Q variables*! However, this will result in Qubo with 33 nodes. 

In [15]:
girls = Qvar(5, "girls")
boys = Qvar(5, "boys")
teachers = Qvar("teachers", 13)
parents = Qvar(2, "parents")
tickets = Qvar("tickets", 78)
distribution = Qequation(tickets).assign( girls + boys + teachers + parents )
Q = distribution.qubo()
nNo = Qanalyzer(Q).nodesNo()
print("number of nodes:", nNo`, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 33 has 2,147,483,714.0 posible solutions


> On my laptop using d5o.Qsolver **the next section of code will take close to 10 minutes** to solve the Qubo with 33 nodes.

In [16]:
start = time.process_time()
d5oSolver = Qsolver(Q)
sample = d5oSolver.solution()
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 599.28125 seconds to solve problem with 33 nodes


In [17]:
distribution.set(sample)
print(distribution.solutions())

girls = 31; boys = 31; teachers = 13; parents = 3; 



> To avoid waiting so long in the future, I will use D-Wave Ocean sampler API to conenct to QACs to process my problem.

In [18]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.cloud.exceptions import SolverNotFoundError

To learn more about how to work with [different topologies](https://docs.ocean.dwavesys.com/en/stable/examples/topology_samplers.html) review D-Wave documentation. In this example we are connecting to both:
- advanatage QAC instance with pegasus topology, and
- 2000Q QAC instance with chimera topology

In [19]:
try:
    qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus', 'qpu': True})
    qpu_2000q = DWaveSampler(solver={'topology__type': 'chimera', 'qpu': True})
    
    qpus = {'Advantage': qpu_advantage, 'DW-2000Q': qpu_2000q}

    print("Connected to Advantage {} and 2000Q {}.".format(qpu_advantage.solver.id, qpu_2000q.solver.id))
except SolverNotFoundError:
    print("Currently a pair of solvers are unavailable for sections comparing QPU technologies. Try those examples later.")

Connected to Advantage Advantage_system1.1 and 2000Q DW_2000Q_6.


Now that we are connected to Advantage and 2000Q QACs, we can start by crating ***advantage*** solver by using D-Wave's composite [embedding](https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/embedding.html) of our Qubo to pegasus topology.

In [20]:
advantage = EmbeddingComposite(qpu_advantage)

For the ***advantage*** solver let prepare arguments for remote processing, i.e. **let set** ***kwargs***.
- ***kwargs*** *represents optional keyword arguments for the sampling method, specified per [solver](https://docs.dwavesys.com/docs/latest/doc_solver_ref.html) in parameters*. 

The arguments used in this example are self explanatory.

> I am founding that ***chain strenght 5*** works well for these kind of problems on pegasus topology.

In [21]:
kwargs = {}
if 'num_reads' in advantage.parameters:
    kwargs['num_reads'] = 100
if 'answer_mode' in advantage.parameters:
    kwargs['answer_mode'] = 'histogram'
if 'chain_strength' in advantage.parameters:
    kwargs['chain_strength'] = 5

In [22]:
start = time.process_time()
sampleset = advantage.sample_qubo(Q, **kwargs)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 0.375 seconds to solve problem with 33 nodes


In [23]:
distribution.set(sample)
print(distribution.solutions())

girls = 31; boys = 31; teachers = 13; parents = 3; 



Running *same kwargs* on **2000Q chimera topology**.

In [24]:
start = time.process_time()
dw2000q = EmbeddingComposite(qpu_2000q)
sampleset = dw2000q.sample_qubo(Q, **kwargs)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 0.296875 seconds to solve problem with 33 nodes


In [25]:
distribution.set(sample)
print(distribution.solutions())

girls = 31; boys = 31; teachers = 13; parents = 3; 



> The same ***chain strenght 5*** produced correct result on **chimera topology** as it did on *pegasus*.

As expected, on both QAC topologies it took a fraction of a second with all latencies of a remote API to solve a Qubo problem with 33 nodes.

In [32]:
factor1 = Qvar(5, "f1")
factor2 = Qvar(5, "f2")
factor3 = Qvar(4, "f3")
product = Qvar("P", 13020)
eProduct = Qequation(product).assign(factor1 * factor2 * factor3)
Q = eProduct.qubo()
nNo = Qanalyzer(Q).nodesNo()
print("number of nodes:", nNo, "has", f"{'{:,}'.format(pow(2,nNo-1)/2 + 2*nNo)}", "possible solutions")

number of nodes: 165 has 1.1692013098647223e+49 posible solutions


In [33]:
start = time.process_time()
sampleset = advantage.sample_qubo(Q, **kwargs)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 9.078125 seconds to solve problem with 165 nodes


In [34]:
eProduct.set(sample)
print(eProduct.solutions())

f1 = 0; f2 = 0; f3 = 0; 



## d5o equations using D-Wave hybrid solver


In [35]:
from dwave.system.samplers import LeapHybridSampler
hybrid = LeapHybridSampler()

In [38]:
start = time.process_time()
sampleset = hybrid.sample_qubo(Q)
samples = [dict(sample) for sample in sampleset.lowest().samples()]
finish = time.process_time()
print("Qsolver takes", finish - start, "seconds to solve problem with", nNo, "nodes")

Qsolver takes 1.203125 seconds to solve problem with 165 nodes


In [39]:
eProduct.set(sample)
print(eProduct.solutions())

f1 = 0; f2 = 0; f3 = 0; 

